In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

names_air=["Airport ID","Name","City","Country","IATA","ICAO","Latitude", 
                                           "Longitude", "Altitude", "Timezone", "DST", 
                                           "Tz database","Type","Source"]
airports=pd.read_csv("E://airports.csv",names=names_air).replace("\\N",np.nan)

In [2]:
routes_names = ["Airline", "Airline ID", "Source airport", "Source airport ID", 
                "Destination airport", "Destination airport ID", 
                "Codeshare", "Stops", "Equipment"]
routes = pd.read_csv('E://routes.csv', index_col = "Airline ID", 
                     header = None, names = routes_names).replace("\\N",np.nan)

In [3]:
US = airports[
    (airports['Country'] == 'United States')&(airports['IATA'].notna())
    ] 
US = US[~US.IATA.isin(['OEL','TZR','CWT'])]

In [4]:
nonUS = airports[
    (airports['Country'] != 'United States')&(airports['IATA'].notna())
    ]

nonUS = nonUS[~nonUS.IATA.isin(['LND','EMP','LKV','NDZ','QQS','BMQ','JON','BCV'])]

In [5]:
USp = US[['Name','Latitude','Longitude']]

routes = routes[routes['Source airport'].isin(nonUS.IATA)]
routes = routes[routes['Destination airport'].isin(US.IATA)]


unstack_table = routes.groupby(['Source airport','Destination airport']).count()\
.drop(['Airline', 'Source airport ID', 'Destination airport ID', 'Codeshare','Equipment'],axis=1)


unstack_table = unstack_table.reset_index()


nonUS_coords = pd.merge(unstack_table,airports,left_on='Source airport',right_on='IATA',how='left')
US_coords = pd.merge(unstack_table,airports,left_on='Destination airport',right_on='IATA',how='left')

unstack_table['start_lat'] = nonUS_coords['Latitude']
unstack_table['start_lon'] = nonUS_coords['Longitude']
unstack_table['end_lat'] = US_coords['Latitude']
unstack_table['end_lon'] = US_coords['Longitude']

In [12]:
US1 = nonUS[nonUS.IATA.isin(unstack_table['Source airport'].unique())]
markers = pd.concat([US,US1])

In [13]:
fig = go.Figure()

flight_paths = []
for i in range(len(unstack_table)):
    fig.add_trace(
        go.Scattergeo(
            lon = [unstack_table['start_lon'][i], unstack_table['end_lon'][i]],
            lat = [unstack_table['start_lat'][i], unstack_table['end_lat'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = float(unstack_table['Stops'][i]) / float(unstack_table['Stops'].max()),
        )
    )

fig.update_layout(
    title_text = 'РАКЕТНАЯ АТАКА ПО США',
    showlegend = False,
    geo = go.layout.Geo(
        scope = 'world',
        #projection=dict( type='equirectangular' ),
            projection=dict( type ="natural earth"),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
        margin={"r":50,"t":50,"l":1,"b":1}
)

fig.add_trace(go.Scattergeo(
    lon = markers['Longitude'],
    lat = markers['Latitude'],
    hoverinfo = 'text',
    text = markers['Name'],
    mode = 'markers',
    marker = dict(
        size = 2,
        color = 'rgb(255, 0, 0)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

fig.show()